In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.float_format',lambda x: '%.4f' % x)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.pyplot import figure
import matplotlib.pylab as pll
from collections import Counter
from datetime import datetime
import time
import random
import math
import os
import itertools
from matplotlib import ticker
import matplotlib.colors as colors
from pylab import rcParams

In [ ]:
slurm1 = pd.read_csv('SlurmPlot2.csv')
slurm1.tail(100)

In [ ]:
slurm1.head(2)

In [ ]:
slurm = slurm1[['UID','JobID','Account','Submittimestamp','Start','End','Eligibletimestamp','Starttimestamp','Endtimestamp','AllocNodes','AllocCPUS','TimelimitRaw', 'NNodes','Submit','State','ExitCode','QOS','QOSRAW','Partition','TRESAllocMem','TRESINMaxUsagemem','TotalCPUtime']]

In [ ]:
slurm = slurm[slurm["Starttimestamp"] != 'Unknown']
slurm = slurm[slurm["Endtimestamp"] != 'Unknown']
slurm['JobLen'] = slurm['Endtimestamp'].astype(float) - slurm['Starttimestamp'].astype(float)
slurm['Totalres'] = (slurm['JobLen']/3600) * slurm['NNodes']
slurm['JobID'] = slurm['JobID'].astype(str)

In [ ]:
discard = ["."]
waittime = slurm[~slurm.JobID.str.contains('|'.join(discard),regex = False)]
waittime['Jcount'] = 1
waittime['Waittime'] = waittime['Starttimestamp'].astype(float) - waittime['Submittimestamp'].astype(float)

In [ ]:
waittime['year'] = pd.DatetimeIndex(waittime['Submit']).year
waittime['month'] = pd.DatetimeIndex(waittime['Submit']).month
waittime['year'] = waittime['year'].apply(str)
waittime['month'] = waittime['month'].apply(str)
waittime['yearmonth'] = pd.to_datetime(waittime['Submit']).dt.to_period('M')

In [ ]:
waittime.head(50)

In [ ]:
waittime = waittime.loc[waittime.JobLen >= 0]

In [ ]:
waittime.info()

## Cluster Usage 

In [ ]:
waittime.head()

In [ ]:
from datetime import datetime as dt
def functioncluster2(account):
    varlist = []
    for a in account:
        temp = waittime.loc[waittime.Account == a]
        temp1 = temp.groupby(['yearmonth'], as_index=False).agg({'Totalres':'sum','Jcount':'sum'})
        print(temp1['yearmonth'])
        xticks = ['2019-3','2019-4','2019-5','2019-6','2019-7','2019-8','2019-9','2019-10','2019-11','2019-12','2020-1','2020-2','2020-3','2020-4','2020-5','2020-6','2020-7','2020-8','2020-9','2020-10','2020-11','2020-12','2021-1','2021-2','2021-3','2021-4','2021-5','2021-6','2021-7','2021-8','2021-9','2021-10','2021-11','2021-12','2022-1']
        xticks1 =  pd.to_datetime(xticks).to_period('M')
        yearmonth = temp1['yearmonth'].tolist()
        for xtick in xticks1:
            if xtick not in yearmonth:
                temp1 = temp1.append({'yearmonth': xtick}, ignore_index=True)
                temp1['Jcount'] = temp1['Jcount'].fillna(0)
                temp1['Totalres'] = temp1['Totalres'].fillna(0)
        temp1 = temp1.sort_values('yearmonth')
        # temp1['Totalres'] = temp1['Totalres']/3600
        temp1['Totalres'] = temp1['Totalres'].astype(int)
        fig = plt.figure() # Create matplotlib figure
        ax = fig.add_subplot(111) 
        bar1 = np.arange(len(temp1.yearmonth))
        ax.bar(bar1, temp1.Totalres, color='g')
        ax.tick_params(axis='x',length=0)
        fig.set_figheight(10)
        fig.set_figwidth(60)
        for p in ax.patches:
            ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.005),fontsize=24,rotation = 90)
        ax.set_xlabel('Months', fontsize=22)
        ax.set_ylabel('Total resources in hours', fontsize=22)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("24")
        for tick in ax.get_xticklabels():
            tick.set_fontsize("0")
        plt.show()
        if not os.path.exists('Cluster_Usage'):
            os.makedirs('Cluster_Usage')
        fig.savefig('Cluster_Usage/{}_Cluster_Usage.png'.format(a),bbox_inches='tight', dpi=100)
                
            

In [ ]:
account = waittime['Account'].unique()
account = account.tolist()
functioncluster2(account)

In [ ]:
from datetime import datetime as dt
def functioncluster(account):
    varlist = []
    for a in account:
        temp = waittime.loc[waittime.Account == a]
        temp1 = temp.groupby(['yearmonth'], as_index=False).agg({'Totalres':'sum','Jcount':'sum'})
        print(temp1['yearmonth'])
        xticks = ['2019-3','2019-4','2019-5','2019-6','2019-7','2019-8','2019-9','2019-10','2019-11','2019-12','2020-1','2020-2','2020-3','2020-4','2020-5','2020-6','2020-7','2020-8','2020-9','2020-10','2020-11','2020-12','2021-1','2021-2','2021-3','2021-4','2021-5','2021-6','2021-7','2021-8','2021-9','2021-10','2021-11','2021-12','2022-1']
        xticks1 =  pd.to_datetime(xticks).to_period('M')
        yearmonth = temp1['yearmonth'].tolist()
        for xtick in xticks1:
            if xtick not in yearmonth:
                temp1 = temp1.append({'yearmonth': xtick}, ignore_index=True)
                temp1['Jcount'] = temp1['Jcount'].fillna(0)
                temp1['Totalres'] = temp1['Totalres'].fillna(0)
        temp1 = temp1.sort_values('yearmonth')
        # temp1['Totalres'] = temp1['Totalres']/3600
        temp1['Totalres'] = temp1['Totalres'].astype(int)
        fig = plt.figure() # Create matplotlib figure
        ax = fig.add_subplot(111)
        width = .4 
        bar1 = np.arange(len(temp1.yearmonth))
        bar2 = [ i + width for i in bar1]
        ax.bar(bar1, temp1.Totalres, width, color='g')
        ax2 = ax.twinx() 
        ax2.bar(bar2, temp1.Jcount,width, color='b')
        ax.set_xticks(bar1 + width/2,temp1.yearmonth)
        fig.set_figheight(10)
        fig.set_figwidth(60)
        for p in ax.patches:
            ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.005),fontsize=22,rotation = 90)
        for p in ax2.patches:
            ax2.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.005),fontsize=22,rotation = 90)
        ax.set_xlabel('Months', fontsize=18)
        ax.set_ylabel('Total resources in seconds', fontsize=18)
        ax2.set_ylabel('total jobs', fontsize=18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("22")
        for tick in ax.get_xticklabels():
            tick.set_fontsize("20")
        for tick in ax2.get_yticklabels():
            tick.set_fontsize("22")
        plt.show()
        if not os.path.exists('Cluster_Usage'):
            os.makedirs('Cluster_Usage')
        fig.savefig('Cluster_Usage/{}_Cluster_Usage.png'.format(a),bbox_inches='tight', dpi=100)
                
            

In [ ]:
account = waittime['Account'].unique()
account = account.tolist()
functioncluster(account)

# report purpose

## For hours

In [ ]:
from datetime import datetime as dt
%precision %.2f
pd.options.display.float_format = '{:,.2f}'.format
def functioncluster3(account):
    varlist = []
    for a in account:
        temp = waittime.loc[waittime.Account == a]
        temp1 = temp.groupby(['yearmonth'], as_index=False).agg({'Totalres':'sum','Jcount':'sum'})
        print(temp1['yearmonth'])
        xticks = ['2019-3','2019-4','2019-5','2019-6','2019-7','2019-8','2019-9','2019-10','2019-11','2019-12','2020-1','2020-2','2020-3','2020-4','2020-5','2020-6','2020-7','2020-8','2020-9','2020-10','2020-11','2020-12','2021-1','2021-2','2021-3','2021-4','2021-5','2021-6','2021-7','2021-8','2021-9','2021-10','2021-11','2021-12','2022-1']
        xticks1 =  pd.to_datetime(xticks).to_period('M')
        yearmonth = temp1['yearmonth'].tolist()
        for xtick in xticks1:
            if xtick not in yearmonth:
                temp1 = temp1.append({'yearmonth': xtick}, ignore_index=True)
                temp1['Jcount'] = temp1['Jcount'].fillna(0)
                temp1['Totalres'] = temp1['Totalres'].fillna(0)
        temp1 = temp1.sort_values('yearmonth')
        temp1['Totalres'] = temp1['Totalres']/3600
        temp1['Totalres'] = temp1['Totalres'].astype(int)
        fig = plt.figure() # Create matplotlib figure
        ax = fig.add_subplot(111)
        ax.set_title("Total resources for jobs every month for the account {} in hours".format(a,size=22))
        width = .4 
        bar1 = np.arange(len(temp1.yearmonth))
        bar2 = [ i + width for i in bar1]
        ax.bar(bar1, temp1.Totalres, width, color='g')
        ax2 = ax.twinx() 
        ax2.bar(bar2, temp1.Jcount,width, color='b')
        ax.set_xticks(bar1 + width/2,temp1.yearmonth)
        fig.set_figheight(10)
        fig.set_figwidth(60)
        for p in ax.patches:
            ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.05),fontsize=22,rotation = 90)
        for p in ax2.patches:
            ax2.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.05),fontsize=22,rotation = 90)
        ax.set_xlabel('Months', fontsize=18)
        ax.set_ylabel('Total resources in hours', fontsize=18)
        ax2.set_ylabel('total jobs', fontsize=18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("22")
        for tick in ax.get_xticklabels():
            tick.set_fontsize("20")
        for tick in ax2.get_yticklabels():
            tick.set_fontsize("22")
        plt.show()
        if not os.path.exists('Cluster_Usage'):
            os.makedirs('Cluster_Usage')
        fig.savefig('Cluster_Usage/{}_Cluster_Usage_hours.png'.format(a),bbox_inches='tight', dpi=100)
                
            

In [ ]:
account = waittime['Account'].unique()
account = account.tolist()
functioncluster3(account)

In [ ]:
def functioncluster2():
    temp = waittime.copy()
    temp1 = temp.groupby(['yearmonth'], as_index=False).agg({'Totalres':'sum','Jcount':'sum'})
    temp1 = temp1.sort_values('yearmonth')
    temp1['Totalres'] = temp1['Totalres']/3600
    temp1['Totalres'] = temp1['Totalres'].astype(int)
    fig = plt.figure() # Create matplotlib figure
    ax = fig.add_subplot(111)
    width = .4 
    bar1 = np.arange(len(temp1.yearmonth))
    bar2 = [ i + width for i in bar1]
    ax.bar(bar1, temp1.Totalres, width, color='g')
    ax2 = ax.twinx() 
    ax2.bar(bar2, temp1.Jcount,width, color='b')
    ax.set_xticks(bar1 + width/2,temp1.yearmonth)
    fig.set_figheight(10)
    fig.set_figwidth(60)
    for p in ax.patches:
        ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.05),fontsize=22,rotation = 90)
    for p in ax2.patches:
        ax2.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.05),fontsize=22,rotation = 90)
    ax.set_xlabel('Months', fontsize=18)
    ax.set_ylabel('Total resources in hours', fontsize=18)
    ax2.set_ylabel('total jobs', fontsize=18)
    for tick in ax.get_yticklabels():
        tick.set_fontsize("22")
    for tick in ax.get_xticklabels():
        tick.set_fontsize("20")
    for tick in ax2.get_yticklabels():
        tick.set_fontsize("22")
    # ax.set(xticks = l.month, xticklabels=month_name)
    plt.show()
    if not os.path.exists('Cluster_Usage'):
        os.makedirs('Cluster_Usage')
    fig.savefig('Cluster_Usage/Cluster_Usage_all.png',bbox_inches='tight', dpi=100)             
            

In [ ]:
functioncluster2()

In [ ]:
def functioncluster3():
    temp = waittime.copy()
    temp1 = temp.groupby(['yearmonth'], as_index=False).agg({'Totalres':'sum','Jcount':'sum'})
    temp1 = temp1.sort_values('yearmonth')
    temp1['Totalres'] = temp1['Totalres'].astype(int)
    
    fig = plt.figure() # Create matplotlib figure
    ax = fig.add_subplot(111)
    width = .4 
    bar1 = np.arange(len(temp1.yearmonth))
    ax.bar(bar1, temp1.Totalres, width, color='g')
    fig.set_figheight(10)
    fig.set_figwidth(60)
    for p in ax.patches:
        ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.05),fontsize=22,rotation = 90)
    ax.set_xlabel('Months', fontsize=18)
    ax.set_ylabel('Total resources in hours', fontsize=18)
    for tick in ax.get_yticklabels():
        tick.set_fontsize("22")
    for tick in ax.get_xticklabels():
        tick.set_fontsize("20")
    # ax.set(xticks = l.month, xticklabels=month_name)
    plt.show()
    if not os.path.exists('Cluster_Usage'):
        os.makedirs('Cluster_Usage')
    fig.savefig('Cluster_Usage/Cluster_Usage_all.png',bbox_inches='tight', dpi=100)             
            

In [ ]:
functioncluster3()